In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import math

In [12]:
#constant

data_raw_frame = 100
opacity = 0.5
contrast_0_to_1 = 0.4

In [10]:
def gradient(lidar_df : pd.DataFrame, contrast_cof : float = contrast_0_to_1) -> list[float]:
    """
    Converts the reflection lidar_df column to a gradient from 0 to 1.

    Parameters
    ----------
    lidar_df : pd.DataFrame
        A lidar dataframe that has a reflection column
    gradient_cof : float
        Float from the segment [0, 1]. The smaller the number, the stronger the contrast
        

    Returns
    -------
    list[float]
        a list containing floats of [0, 1] that are responsible for the color and opacity in RGB
    """
    gradient_list : list[float] = []

    #хотим выбрать степенную функцию, для которой средний градиент переходит в 0.5
    mean_gradient : float= sum(lidar_df["r, (reflectance)"]) / (len(lidar_df["r, (reflectance)"]))
    power_normalization : float = math.log(2) / math.log(mean_gradient) 


    for reflect in lidar_df["r, (reflectance)"]:
        color : float = (reflect / 255) ** power_normalization

        #переводим в [-1, 1] берём корень и переаодим обратно в [0, 1]
        if((2 * color - 1) > 0):
            color_normalize : float = 0.5 * ((abs((2 * color - 1)) ** contrast_0_to_1) + 1)
        else:
            color_normalize : float  = 0.5 * (-(abs((2 * color - 1)) ** contrast_0_to_1) + 1)
        
        gradient_list.append((color_normalize, 0, 1 - color_normalize, opacity))
    return gradient_list

In [ ]:
for frame_num in tqdm(range(0, data_raw_frame)):
    frame_radar : pd.DataFrame = pd.read_csv(f"data/processed data/radar_data_{frame_num}.csv")
    frame_lidar : pd.DataFrame = pd.read_csv(f"data/processed data/lidar_data_{frame_num}.csv")

    plt.figure(figsize = (22, 10))
    plt.xlim([-110, 110])
    plt.ylim([-50, 50])
    
    plt.scatter(frame_lidar["X, (m)"], frame_lidar["Y, (m)"], s=1, c=gradient(frame_lidar))
    plt.scatter(frame_radar[frame_radar["QPDH0"] < 0.3]["X, (m)"], frame_radar[frame_radar["QPDH0"] < 0.3]["Y, (m)"], s=1, c = "green")

    plt.title(f"Frame {frame_num}")
    plt.savefig(f"data/radar_lidar_png/Frame_{frame_num}.png")
    plt.close()